In [1]:
%load_ext autoreload
%autoreload 2
import torch

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from classification.models.M5 import M5PLModule

In [3]:
loaded_dict = torch.load("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p")
hparams = loaded_dict["hparams"]
hparams['batch_size'] = 30

model = M5PLModule(hparams)
model.model.load_state_dict(loaded_dict["state_dict"])

<All keys matched successfully>

In [4]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

In [5]:
datasetHandler.load(model, 'validation')

Loading cached validation data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_8k/


In [8]:
from attacks.pgd import ProjectedGradientDescent

epsilons = [0.02]
#epsilons = [0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5]

for eps in epsilons:
    attack_parameters = {'epsilon': eps, 'num_iter': 1}
    valid_loader = model.val_dataloader()
    fgsm = ProjectedGradientDescent(model, valid_loader, attack_parameters, save_samples=False)
    fgsm.attack(cudnn_RNN_backward_bug=False)

100%|██████████| 57/57 [00:03<00:00, 15.36it/s]


In [9]:
fgsm.report()

Attack-Parameters:	{'epsilon': 0.02, 'num_iter': 1}
Early stopping: 	False (-1)

Successfully attacked:	1032
Total attacked: 	1420
Total processed:	1687

Success-Rate: 		0.73
Perturbed Accurracy: 	0.23


{'success_rate': 0.7267605633802817, 'acc': 0.22999407231772376}